# Integrated Arduino Opta Controller Test Notebook

This notebook provides a comprehensive testing interface for the integrated Arduino Opta controller system.

## Supported Devices:
- **Relays**: REL_01 to REL_04 (digital pin control)
- **VICI Valves**: VICI_01, VICI_02, etc. (RS485 communication)
- **Masterflex Pumps**: MFLEX_01, MFLEX_02, etc. (RS485 communication)

## Command Protocol:
```
DEVICE_ID:COMMAND[:PARAM1[:PARAM2]]
```

## Examples:
- `REL_01:ON` - Turn on Relay 1
- `VICI_01:GOTO:A` - Move VICI valve to position A
- `MFLEX_01:SPEED:100.0:+` - Set Masterflex pump speed to 100 RPM forward


## Setup and Connection

In [ ]:
# Import required libraries
import sys
import os
import time
import pandas as pd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Add the current directory to Python path to import our controller
sys.path.append('.')

from integrated_opta_client import IntegratedOptaController

print("Libraries imported successfully!")
print("Ready to connect to Arduino Opta Controller")

In [ ]:
# Configuration - Update these values for your setup
SERIAL_PORT = 'COM3'  # Change to your Arduino port (Windows: COM3, Linux/Mac: /dev/ttyUSB0)
BAUDRATE = 115200

# Connect to the Arduino Opta Controller
try:
    controller = IntegratedOptaController(port=SERIAL_PORT, baudrate=BAUDRATE)
    
    if controller.connected:
        print(f"✅ Successfully connected to Arduino Opta on {SERIAL_PORT}")
        
        # Display system information
        print("\n📊 System Information:")
        controller.system_info()
    else:
        print(f"❌ Failed to connect to Arduino Opta on {SERIAL_PORT}")
        print("Please check:")
        print("- Arduino is connected and powered")
        print("- Correct serial port in SERIAL_PORT variable")
        print("- Arduino sketch is uploaded and running")
        
except Exception as e:
    print(f"❌ Connection error: {e}")
    controller = None

## System Status and Device Discovery

In [ ]:
# Get overall system status
if controller and controller.connected:
    print("📋 Current System Status:")
    status = controller.get_status()
    if status:
        print(f"Raw status: {status}")
        
        # Parse and display device statuses nicely
        if status.startswith("DATA:"):
            device_statuses = status[5:].strip()
            if device_statuses:
                devices = device_statuses.split(", ")
                print("\n📱 Individual Device Status:")
                for device in devices:
                    print(f"  • {device}")
            else:
                print("No devices found or all devices offline")
    else:
        print("❌ No response from system status command")
else:
    print("❌ Controller not connected. Please run the connection cell above.")

## Relay Control Tests

In [ ]:
# Interactive Relay Control
def test_relay(relay_number, action):
    """Test relay control with user-friendly interface."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    relay_id = f"REL_{relay_number:02d}"
    print(f"🔌 Testing {relay_id} - Action: {action}")
    
    if action == "ON":
        response = controller.relay_on(relay_id)
    elif action == "OFF":
        response = controller.relay_off(relay_id)
    elif action == "TOGGLE":
        response = controller.relay_toggle(relay_id)
    else:
        print(f"❌ Unknown action: {action}")
        return
    
    if response:
        if response.startswith("OK:"):
            print(f"✅ {response}")
        elif response.startswith("ERROR:"):
            print(f"❌ {response}")
        else:
            print(f"📝 {response}")
    else:
        print("❌ No response received")

# Create interactive widget for relay control
relay_widget = interact_manual(test_relay,
    relay_number=widgets.IntSlider(min=1, max=4, value=1, description='Relay:'),
    action=widgets.Dropdown(options=['ON', 'OFF', 'TOGGLE'], description='Action:')
)

print("🎛️ Use the controls above to test relay operations")

In [ ]:
# Relay Sequence Test - All relays ON then OFF
def relay_sequence_test():
    """Test all relays in sequence."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print("🔄 Starting relay sequence test...")
    
    # Turn all relays ON
    print("\n📢 Turning all relays ON:")
    for i in range(1, 5):
        relay_id = f"REL_{i:02d}"
        response = controller.relay_on(relay_id)
        print(f"  {relay_id}: {response}")
        time.sleep(0.5)
    
    print("\n⏱️ Waiting 3 seconds...")
    time.sleep(3)
    
    # Turn all relays OFF
    print("\n📢 Turning all relays OFF:")
    for i in range(1, 5):
        relay_id = f"REL_{i:02d}"
        response = controller.relay_off(relay_id)
        print(f"  {relay_id}: {response}")
        time.sleep(0.5)
    
    print("\n✅ Relay sequence test completed")

# Button to run sequence test
sequence_button = widgets.Button(description="Run Relay Sequence Test", button_style='info')
sequence_button.on_click(lambda b: relay_sequence_test())
display(sequence_button)

## VICI Valve Control Tests

In [ ]:
# Interactive VICI Valve Control
def test_vici_valve(valve_id, command, parameter=None):
    """Test VICI valve control with user-friendly interface."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print(f"🚰 Testing {valve_id} - Command: {command}")
    
    if command == "GOTO" and parameter:
        response = controller.vici_goto_position(valve_id, parameter)
    elif command == "TOGGLE":
        response = controller.vici_toggle(valve_id)
    elif command == "HOME":
        response = controller.vici_home(valve_id)
    elif command == "POSITION":
        response = controller.vici_get_position(valve_id)
    elif command == "STATUS":
        response = controller.vici_get_status(valve_id)
    elif command == "CW":
        response = controller.vici_cw(valve_id)
    elif command == "CCW":
        response = controller.vici_ccw(valve_id)
    else:
        print(f"❌ Unknown command: {command}")
        return
    
    if response:
        if response.startswith("OK:"):
            print(f"✅ {response}")
        elif response.startswith("ERROR:"):
            print(f"❌ {response}")
        elif response.startswith("DATA:"):
            print(f"📊 {response}")
        else:
            print(f"📝 {response}")
    else:
        print("❌ No response received")

# Create interactive widget for VICI valve control
vici_widget = interactive(test_vici_valve,
    valve_id=widgets.Dropdown(options=['VICI_01', 'VICI_02'], description='Valve:'),
    command=widgets.Dropdown(options=['GOTO', 'TOGGLE', 'HOME', 'POSITION', 'STATUS', 'CW', 'CCW'], description='Command:'),
    parameter=widgets.Text(description='Position:', placeholder='A, B, or number')
)

display(vici_widget)

In [ ]:
# VICI Valve Cycle Test
def vici_cycle_test(valve_id='VICI_01', cycles=3, delay=2.0):
    """Run automated cycle test for VICI valve."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print(f"🔄 Starting VICI valve cycle test: {valve_id}")
    print(f"Cycles: {cycles}, Delay: {delay}s")
    
    success = controller.valve_cycle_test(valve_id, cycles, delay)
    
    if success:
        print("✅ VICI valve cycle test completed successfully")
    else:
        print("❌ VICI valve cycle test failed")

# Create interactive widget for cycle test
vici_cycle_widget = interactive(vici_cycle_test,
    valve_id=widgets.Dropdown(options=['VICI_01', 'VICI_02'], value='VICI_01', description='Valve:'),
    cycles=widgets.IntSlider(min=1, max=10, value=3, description='Cycles:'),
    delay=widgets.FloatSlider(min=0.5, max=5.0, value=2.0, step=0.5, description='Delay (s):')
)

display(vici_cycle_widget)

## Masterflex Pump Control Tests

In [ ]:
# Masterflex Pump Initialization
def initialize_pump(pump_id='MFLEX_01'):
    """Initialize Masterflex pump communication."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print(f"🔧 Initializing pump {pump_id}...")
    response = controller.masterflex_init(pump_id)
    
    if response:
        if response.startswith("OK:"):
            print(f"✅ {response}")
            
            # Enable remote mode
            print("📡 Enabling remote mode...")
            remote_response = controller.masterflex_remote_mode(pump_id)
            print(f"Remote mode: {remote_response}")
            
            # Get status
            print("📊 Getting pump status...")
            status_response = controller.masterflex_get_status(pump_id)
            print(f"Status: {status_response}")
            
        else:
            print(f"❌ Initialization failed: {response}")
    else:
        print("❌ No response from pump initialization")

# Button to initialize pump
pump_init_widget = interactive(initialize_pump,
    pump_id=widgets.Dropdown(options=['MFLEX_01', 'MFLEX_02'], value='MFLEX_01', description='Pump:')
)
display(pump_init_widget)

In [ ]:
# Interactive Masterflex Pump Control
def test_masterflex_pump(pump_id, command, rpm=None, revolutions=None, direction='+'):
    """Test Masterflex pump control with user-friendly interface."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print(f"🔄 Testing {pump_id} - Command: {command}")
    
    if command == "SPEED" and rpm is not None:
        response = controller.masterflex_set_speed(pump_id, rpm, direction)
    elif command == "START":
        response = controller.masterflex_start(pump_id)
    elif command == "STOP":
        response = controller.masterflex_stop(pump_id)
    elif command == "REVOLUTIONS" and revolutions is not None:
        response = controller.masterflex_set_revolutions(pump_id, revolutions)
    elif command == "STATUS":
        response = controller.masterflex_get_status(pump_id)
    elif command == "REMOTE":
        response = controller.masterflex_remote_mode(pump_id)
    elif command == "LOCAL":
        response = controller.masterflex_local_mode(pump_id)
    else:
        print(f"❌ Unknown command or missing parameters: {command}")
        return
    
    if response:
        if response.startswith("OK:"):
            print(f"✅ {response}")
        elif response.startswith("ERROR:"):
            print(f"❌ {response}")
        elif response.startswith("DATA:"):
            print(f"📊 {response}")
        else:
            print(f"📝 {response}")
    else:
        print("❌ No response received")

# Create interactive widget for Masterflex pump control
pump_widget = interactive(test_masterflex_pump,
    pump_id=widgets.Dropdown(options=['MFLEX_01', 'MFLEX_02'], description='Pump:'),
    command=widgets.Dropdown(options=['SPEED', 'START', 'STOP', 'REVOLUTIONS', 'STATUS', 'REMOTE', 'LOCAL'], description='Command:'),
    rpm=widgets.FloatText(value=50.0, description='RPM:', disabled=False),
    revolutions=widgets.FloatText(value=10.0, description='Revolutions:', disabled=False),
    direction=widgets.Dropdown(options=['+', '-'], value='+', description='Direction:')
)

display(pump_widget)

In [ ]:
# Automated Pump Sequence Test
def pump_sequence_test(pump_id='MFLEX_01', rpm=100.0, revolutions=5.0, direction='+'):
    """Run automated pump sequence test."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print(f"🧪 Starting pump sequence test: {pump_id}")
    print(f"Parameters: {rpm} RPM, {revolutions} revolutions, direction {direction}")
    
    success = controller.run_pump_sequence(pump_id, rpm, revolutions, direction)
    
    if success:
        print("✅ Pump sequence started successfully")
        print("💡 Monitor pump status manually or check physical pump operation")
    else:
        print("❌ Pump sequence failed to start")

# Create interactive widget for pump sequence test
pump_sequence_widget = interactive(pump_sequence_test,
    pump_id=widgets.Dropdown(options=['MFLEX_01', 'MFLEX_02'], value='MFLEX_01', description='Pump:'),
    rpm=widgets.FloatSlider(min=1.0, max=300.0, value=100.0, step=1.0, description='RPM:'),
    revolutions=widgets.FloatSlider(min=0.1, max=50.0, value=5.0, step=0.1, description='Revolutions:'),
    direction=widgets.Dropdown(options=['+', '-'], value='+', description='Direction:')
)

display(pump_sequence_widget)

## Emergency Controls and Safety

In [ ]:
# Emergency Stop Function
def emergency_stop():
    """Emergency stop all devices."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print("🚨 EMERGENCY STOP ACTIVATED")
    controller.emergency_stop()
    print("✅ All devices stopped")

# Create emergency stop button with warning style
emergency_button = widgets.Button(
    description="🚨 EMERGENCY STOP",
    button_style='danger',
    layout=widgets.Layout(width='200px', height='50px')
)
emergency_button.on_click(lambda b: emergency_stop())

print("⚠️ EMERGENCY CONTROLS ⚠️")
display(emergency_button)
print("Use this button to immediately stop all pumps and turn off all relays")

## Advanced Testing and Custom Commands

In [ ]:
# Custom Command Interface
def send_custom_command(command):
    """Send a custom command to the Arduino controller."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    if not command.strip():
        print("❌ Please enter a command")
        return
    
    print(f"📤 Sending command: {command}")
    response = controller.send_command(command)
    
    if response:
        print(f"📥 Response: {response}")
    else:
        print("❌ No response received")

# Create interactive widget for custom commands
custom_command_widget = interactive(send_custom_command,
    command=widgets.Text(
        description='Command:',
        placeholder='Enter custom command (e.g., REL_01:ON)',
        layout=widgets.Layout(width='400px')
    )
)

display(custom_command_widget)

print("\n💡 Examples of custom commands:")
print("  • STATUS - Get all device statuses")
print("  • HELP - Get list of available commands")
print("  • REL_03:TOGGLE - Toggle relay 3")
print("  • VICI_01:POSITION - Get VICI valve position")
print("  • MFLEX_01:SPEED:150.0:+ - Set pump speed")

## Data Logging and Monitoring

In [ ]:
# System Monitoring Function
def monitor_system(duration=30, interval=2):
    """Monitor system status for a specified duration."""
    if not controller or not controller.connected:
        print("❌ Controller not connected")
        return
    
    print(f"📊 Starting system monitoring for {duration} seconds (interval: {interval}s)")
    print("Use Kernel > Interrupt to stop monitoring early")
    
    start_time = time.time()
    log_data = []
    
    try:
        while (time.time() - start_time) < duration:
            timestamp = time.time() - start_time
            
            # Get system status
            status = controller.get_status()
            
            # Log data
            log_entry = {
                'timestamp': f"{timestamp:.1f}s",
                'status': status if status else "No response"
            }
            log_data.append(log_entry)
            
            # Display current status
            clear_output(wait=True)
            print(f"📊 System Monitoring - Elapsed: {timestamp:.1f}s / {duration}s")
            print(f"Current Status: {status}")
            print(f"Samples collected: {len(log_data)}")
            
            time.sleep(interval)
    
    except KeyboardInterrupt:
        print("\n⏹️ Monitoring stopped by user")
    
    # Display summary
    print(f"\n✅ Monitoring completed. Collected {len(log_data)} samples")
    
    if log_data:
        # Create DataFrame for better display
        df = pd.DataFrame(log_data)
        print("\n📋 Monitoring Summary:")
        display(df.tail(10))  # Show last 10 entries
    
    return log_data

# Create interactive widget for monitoring
monitor_widget = interactive(monitor_system,
    duration=widgets.IntSlider(min=10, max=300, value=30, description='Duration (s):'),
    interval=widgets.FloatSlider(min=0.5, max=10.0, value=2.0, step=0.5, description='Interval (s):')
)

display(monitor_widget)

## Cleanup and Disconnection

In [ ]:
# Safe disconnection
def safe_disconnect():
    """Safely disconnect from the Arduino controller."""
    if controller and controller.connected:
        print("🔄 Performing safe shutdown...")
        
        # Emergency stop to ensure all devices are off
        controller.emergency_stop()
        
        # Disconnect
        controller.disconnect()
        print("✅ Safely disconnected from Arduino Opta Controller")
    else:
        print("ℹ️ No active connection to disconnect")

# Create disconnect button
disconnect_button = widgets.Button(
    description="🔌 Safe Disconnect",
    button_style='warning',
    layout=widgets.Layout(width='150px')
)
disconnect_button.on_click(lambda b: safe_disconnect())

print("🔌 Connection Management:")
display(disconnect_button)
print("Always use safe disconnect before closing the notebook")

## Troubleshooting Guide

### Common Issues:

1. **Connection Failed**
   - Check serial port name (Windows: COM3, Linux/Mac: /dev/ttyUSB0)
   - Ensure Arduino is connected and powered
   - Verify Arduino sketch is uploaded and running
   - Try different baud rate (115200 is default)

2. **No Response from Commands**
   - Check if Arduino serial monitor shows incoming commands
   - Verify command format: DEVICE_ID:COMMAND[:PARAMS]
   - Check device is initialized (especially for Masterflex pumps)

3. **Device Not Found Errors**
   - Check device ID format (REL_01, VICI_01, MFLEX_01)
   - Verify device is configured in Arduino sketch
   - Check physical connections to devices

4. **RS485 Communication Issues**
   - Check RS485 wiring (A+, B-, GND)
   - Verify device IDs match configuration
   - Check termination resistors if using long cables
   - For Masterflex: Initialize pump first with MFLEX_01:INIT

### Device-Specific Notes:

- **Relays**: Direct digital pin control, should respond immediately
- **VICI Valves**: RS485 at 9600 baud, need proper device ID (default '3')
- **Masterflex Pumps**: RS485 at 4800 baud with 7O1, require initialization sequence

### Next Steps for Peptide Synthesizer Integration:

1. Test all devices individually using this notebook
2. Create operation sequences that match your synthesis protocols
3. Implement CSV command file parsing in Python
4. Add error handling and recovery mechanisms
5. Integrate with peptide synthesis software